In [14]:
import concurrent.futures
import pandas as pd
def parse_variables(lines):
    variables = {}
    for line in lines:
        if line.startswith("PROBLEM NAME:"):
            variables['problem_name'] = line.split(':')[1].strip()
        elif line.startswith("KNAPSACK DATA TYPE:"):
            variables['knapsack_data_type'] = line.split(':')[1].strip()
        elif line.startswith("DIMENSION:"):
            variables['dimension'] = int(line.split(':')[1].strip())
        elif line.startswith("NUMBER OF ITEMS:"):
            variables['num_items'] = int(line.split(':')[1].strip())
        elif line.startswith("CAPACITY OF KNAPSACK:"):
            variables['knapsack_capacity'] = int(line.split(':')[1].strip())
        elif line.startswith("MIN SPEED:"):
            variables['min_speed'] = float(line.split(':')[1].strip())
        elif line.startswith("MAX SPEED:"):
            variables['max_speed'] = float(line.split(':')[1].strip())
        elif line.startswith("RENTING RATIO:"):
            variables['renting_ratio'] = float(line.split(':')[1].strip())
        elif line.startswith("EDGE_WEIGHT_TYPE:"):
            variables['edge_weight_type'] = line.split(':')[1].strip()
    return variables

def parse_node_coord(lines, start_line, dimension):
    data = []
    for line in lines[start_line:start_line + dimension]:
        parts = line.split()
        index = int(parts[0])
        x = float(parts[1])
        y = float(parts[2])
        data.append({'Index': index, 'X': x, 'Y': y})
    return pd.DataFrame(data)

def parse_item_section(lines, start_line, dimension):
    data = []
    for line in lines[start_line:start_line + dimension]:
        index, profit, weight, assigned_node = map(int, line.split())
        data.append({'Index': index, 'Profit': profit, 'Weight': weight, 'Assigned_Node': assigned_node})
    return pd.DataFrame(data)

def parse_file_parallel(lines, dimension):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_node_coord = executor.submit(parse_node_coord, lines, 10, dimension)
        future_profit_matrix = executor.submit(parse_item_section, lines, 10 + dimension + 1, dimension)

    node_coord = future_node_coord.result()
    profit_matrix = future_profit_matrix.result()

    return node_coord, profit_matrix


# Example usage:

with open('test-example-n4.txt', 'r') as file:
    lines = file.readlines()

variables = parse_variables(lines)
node_coord, profit_matrix = parse_file_parallel(lines, variables['dimension'])

print("Variables:", variables)
print("Node Coord:", node_coord)
print("Profit Matrix:", profit_matrix)


Variables: {'problem_name': 'Test', 'knapsack_data_type': 'unknown', 'dimension': 4, 'num_items': 3, 'knapsack_capacity': 80, 'min_speed': 0.1, 'max_speed': 1.0, 'renting_ratio': 1.516, 'edge_weight_type': 'CEIL_2D'}
Node Coord:    Index    X    Y
0      1  0.0  0.0
1      2  4.0  0.0
2      3  8.0  3.0
3      4  0.0  3.0
Profit Matrix:    Index  Profit  Weight  Assigned_Node
0      2      34      30              2
1      3      40      40              3
2      4      25      21              4


In [15]:
variables

{'problem_name': 'Test',
 'knapsack_data_type': 'unknown',
 'dimension': 4,
 'num_items': 3,
 'knapsack_capacity': 80,
 'min_speed': 0.1,
 'max_speed': 1.0,
 'renting_ratio': 1.516,
 'edge_weight_type': 'CEIL_2D'}

In [16]:
node_coord

,Index,X,Y
0,1,0.0,0.0
1,2,4.0,0.0
2,3,8.0,3.0
3,4,0.0,3.0


In [17]:
profit_table=profit_matrix

In [18]:
import numpy as np

def calculate_distance_matrix(node_coord):
    coordinates = node_coord[['X', 'Y']].values
    x, y = np.meshgrid(coordinates[:, 0], coordinates[:, 1])

    distance_matrix = np.sqrt((x - x.T)**2 + (y - y.T)**2)
    
    return distance_matrix


In [19]:
distance_matrix=calculate_distance_matrix(node_coord)

In [20]:
distance_matrix

array([[0.        , 4.        , 8.54400375, 3.        ],
       [4.        , 0.        , 5.        , 5.        ],
       [8.54400375, 5.        , 0.        , 8.        ],
       [3.        , 5.        , 8.        , 0.        ]])

In [21]:
profit_table

,Index,Profit,Weight,Assigned_Node
0,2,34,30,2
1,3,40,40,3
2,4,25,21,4


In [26]:
packing_plan_example=[1,0,1]


In [27]:
profit_table['Picked'] = packing_plan_example


In [31]:
profit_table.to_clipboard()

In [22]:
pd.DataFrame(distance_matrix)

,0,1,2,3
0,0.000000,4.0,8.544004,3.0
1,4.000000,0.0,5.000000,5.0
2,8.544004,5.0,0.000000,8.0
3,3.000000,5.0,8.000000,0.0


In [23]:
pd.DataFrame(calculate_distance_matrix(node_coord))

,0,1,2,3
0,0.000000,4.0,8.544004,3.0
1,4.000000,0.0,5.000000,5.0
2,8.544004,5.0,0.000000,8.0
3,3.000000,5.0,8.000000,0.0


In [24]:
tour_example=[1,3,2,4]
packing_plan_example=[1,0,1]

In [80]:
variables

{'dimension': 4,
 'num_items': 3,
 'capacity': 80,
 'min_speed': 0.1,
 'max_speed': 1.0,
 'renting_ratio': 1.516,
 'edge_weight_type': 'CEIL_2D'}

In [88]:
def calculate_current_velocity(weight, capacity, min_speed, max_speed):
    if weight <= capacity:
        return max_speed - (weight / capacity) * (max_speed - min_speed)
    else:
        return min_speed


In [102]:
def calculate_cost(tour, packing_plan, variables,distance_matrix,profit_table):
    dimension = variables['dimension']
    max_speed = variables['max_speed']

    current_weight = 0
    total_time = 0
    current_velocity = max_speed

    for i in range(dimension):
        # Calculate the distance between consecutive cities
        distance = distance_matrix[tour[i % dimension]][tour[(i + 1) % dimension]]

        # Calculate the time taken to travel the distance
        time_to_travel = distance / current_velocity
        total_time += time_to_travel

        # Update current weight based on the packing plan
        current_weight += packing_plan[i % dimension] * profit_table['Weight'][i]

        # Calculate the current velocity
        current_velocity = calculate_current_velocity(current_weight, variables['capacity'],
                                                      variables['min_speed'], variables['max_speed'])

    return total_time


In [103]:
calculate_cost(tour=tour_example,packing_plan=packing_plan_example,variables=variables,distance_matrix=distance_matrix,profit_table=profit_table)

IndexError: index 4 is out of bounds for axis 0 with size 4

In [81]:
def calculate_cost(tour, packing_plan, min_speed, max_speed, renting_ratio, distance_matrix, profit_table):
        total_time = 0.0
        current_weight = 0

        # Initial velocity
        current_velocity = max_speed

        for i in range(len(tour)):
            current_city = tour[i]

            # Calculate the travel time to the next city
            if i < len(tour) - 1:
                next_city = tour[i + 1]
                travel_time = distance_matrix[current_city][next_city] / current_velocity
                total_time += travel_time

            # Update the knapsack weight and velocity based on packing plan
            if packing_plan[current_city - 1] == 1:
                current_weight += profit_table[current_city - 1]['Weight']
                current_velocity = max(min_speed, current_velocity - renting_ratio * current_weight)

        return total_time

In [82]:
variables

{'dimension': 4,
 'num_items': 3,
 'capacity': 80,
 'min_speed': 0.1,
 'max_speed': 1.0,
 'renting_ratio': 1.516,
 'edge_weight_type': 'CEIL_2D'}

In [85]:
calculate_cost(tour_example, packing_plan_example,
                                                variables['min_speed'],
                                                variables['max_speed'],
                                                variables['renting_ratio'],
                                                distance_matrix,
                                                profit_table)

KeyError: 0

In [ ]:
calculate_cost(tour_example, packing_plan_example,
                                                min_speed,
                                                max_speed,
                                                renting_ratio,
                                                distance_matrix,
                                                profit_table)

In [58]:
class TravellingThief:
    def __init__(self, population_size, dimension, num_items, capacity, min_speed, max_speed, renting_ratio, edge_weight_type, distance_matrix, profit_table):
        self.population_size = population_size
        self.dimension = dimension
        self.num_items = num_items
        self.capacity = capacity
        self.min_speed = min_speed
        self.max_speed = max_speed
        self.renting_ratio = renting_ratio
        self.edge_weight_type = edge_weight_type
        self.distance_matrix = distance_matrix
        self.profit_table = profit_table

    @staticmethod
    def calculate_cost(tour, packing_plan, min_speed, max_speed, renting_ratio, distance_matrix, profit_table):
        total_time = 0.0
        current_weight = 0

        # Initial velocity
        current_velocity = max_speed

        for i in range(len(tour)):
            current_city = tour[i]

            # Calculate the travel time to the next city
            if i < len(tour) - 1:
                next_city = tour[i + 1]
                travel_time = distance_matrix[current_city][next_city] / current_velocity
                total_time += travel_time

            # Update the knapsack weight and velocity based on packing plan
            if packing_plan[current_city - 1] == 1:
                current_weight += profit_table[current_city - 1]['Weight']
                current_velocity = max(min_speed, current_velocity - renting_ratio * current_weight)

        return total_time

# Example usage
travelling_thief_instance = TravellingThief(population_size=4, dimension=280, num_items=279, capacity=25936,
                                            min_speed=0.1, max_speed=1.0, renting_ratio=5.61, edge_weight_type='CEIL_2D',
                                            distance_matrix=distance_matrix, profit_table=profit_matrix)

tour_example = [1, 3, 2, 4]
packing_plan_example = [1, 0, 1]

cost = travelling_thief_instance.calculate_cost(tour_example, packing_plan_example,
                                                travelling_thief_instance.min_speed,
                                                travelling_thief_instance.max_speed,
                                                travelling_thief_instance.renting_ratio,
                                                travelling_thief_instance.distance_matrix,
                                                travelling_thief_instance.profit_table)

print(f"Cost for the tour and packing plan: {cost}")


KeyError: 0

In [1]:
import numpy as np

# Given vector
genotype_vector = [0.5, 0.1, 0.8, 0.6, 0.1, 0.9]

# Determine the length of the tour (n) and the length of the packing plan (m)
n = 4  # Assuming the tour has 4 cities
m = 3  # Assuming there are 3 items in the packing plan

# Split the genotype vector into two parts
tour_part = np.array(genotype_vector[:n-1])
packing_plan_part = np.array(genotype_vector[n-1:])

# Decode the tour part to a permutation vector
tour_permutation = np.argsort(tour_part) + 1
tour_permutation = np.insert(tour_permutation, 0, 1)  # Insert the starting city 1

# Decode the packing plan part to a binary decision vector
packing_decision_vector = (packing_plan_part > 0.5).astype(int)

# Display the results
print("Tour Phenotype:", tour_permutation)
print("Packing Plan Phenotype:", packing_decision_vector)


Tour Phenotype: [1 2 1 3]
Packing Plan Phenotype: [1 0 1]


In [2]:
        print(i,current_weight,current_velocity,distance,time_to_travel)


NameError: name 'variables' is not defined

In [13]:
packing_plan


NameError: name 'packing_plan' is not defined